In [1]:
import pandas as pd
#import joblib
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
#from costcla.models import CostSensitiveDecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

C:\Users\DELL\Anaconda3\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('D:\\Data Storm 2.0\\Hotel-A-train.csv')

In [22]:
#label encoding categorical variables
obj_df = new_df.select_dtypes(include=['object']).copy()
obj_df.head()

,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion
0,Latino,Grad,<25K,North,City Hotel,BB,No,No,No Deposit,Online,Yes,Check-In,Yes
1,Latino,Mid-School,50K -- 100K,East,City Hotel,BB,No,No,Refundable,Online,Yes,Check-In,No
2,caucasian,Grad,<25K,East,City Hotel,BB,No,No,No Deposit,Online,Yes,Check-In,No
3,African American,College,>100K,South,Airport Hotels,BB,No,No,Refundable,Agent,Yes,Check-In,Yes
4,Latino,High-School,25K --50K,East,Resort,BB,No,No,No Deposit,Direct,No,Check-In,Yes


In [5]:
label = LabelEncoder()
for col in obj_df.columns:
    new_df[col] = label.fit_transform(new_df[col])
    #return train

In [4]:
#Drop unnecessary variables
new_df = df.drop(['Reservation-id','Gender','Expected_checkin','Expected_checkout','Booking_date','Final Rate','Previous_Cancellations','Use_Promotion','Visted_Previously'],
                 axis=1)

In [65]:
x_train = new_df.drop(['Reservation_Status'], axis=1)

In [66]:
y_train = new_df['Reservation_Status']
y_train = y_train+1

In [8]:
val = pd.read_csv('D:\\Data Storm 2.0\\Hotel-A-validation.csv')
val.columns

Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
       'Expected_checkout', 'Booking_date',
       'time between booking date and check in date', 'time of stay', 'Adults',
       'Children', 'Babies', 'Meal_Type', 'Visted_Previously',
       'Previous_Cancellations', 'Deposit_type', 'Booking_channel',
       'Required_Car_Parking', 'Reservation_Status', 'Use_Promotion',
       'Discount_Rate', 'Room_Rate'],
      dtype='object')

In [9]:
for col in obj_df.columns:
    val[col] = label.fit_transform(val[col])

In [67]:
x_val = val.drop(['Reservation-id','Gender','Expected_checkin','Expected_checkout','Booking_date','Previous_Cancellations','Use_Promotion','Visted_Previously',
                 'Reservation_Status'], axis=1)
y_val = val['Reservation_Status']

In [11]:
y_val = y_val+1

DECISION TREE CLASSIFIER

In [68]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
y_pred_tree = clf.predict(x_val)

In [69]:
dt_f1_score = f1_score(y_val, y_pred_tree, average='weighted')
dt_f1_score

0.1979234292050526

In [19]:
### Fitting XGBoost Model
xgb = XGBClassifier(learning_rate=0.3775, max_depth=5, n_estimators=100, random_state=2)
xgb.fit(x_train,y_train)

## Making predctions
xgb_predict = xgb.predict(x_val)


In [20]:
## F1 Score
xgb_fscore = f1_score(y_val,xgb_predict,average='weighted')
print(xgb_fscore)

0.43733684143945756


In [15]:
#### Parameter tuning
params_grid = {
    'max_depth':[1,2,3,4,5,6],
    'n_estimators':[5,10,25,50,100],
    'learning_rate': np.linspace(0.01,0.5,num=8,endpoint=False)
}

In [16]:
params_fixd = {
    'objective': 'multi:softmax',
}

In [17]:
best_grid = GridSearchCV(
    estimator=XGBClassifier(**params_fixd,random_state=2),
#    estimator=GradientBoostingClassifier(random_state=2),
#    estimator=AdaBoostClassifier(random_state=2),
    param_grid = params_grid,
    scoring='f1_weighted'
)

In [18]:
## Parameter tuning for extreme gradient boosting XGBoost
best_grid.fit(x_train,y_train)
print(best_grid.best_score_,best_grid.best_params_)

0.6738298066349822 {'learning_rate': 0.3775, 'max_depth': 5, 'n_estimators': 100}


In [33]:
#loading the test set
test = pd.read_csv('D:\\Data Storm 2.0\\Hotel-A-test.csv')

In [34]:
#Label encoding test set
for col in obj_df.columns.drop('Reservation_Status'):
    test[col] = label.fit_transform(test[col])

In [35]:
x_test = test.drop(['Reservation-id','Gender','Expected_checkin','Expected_checkout','Booking_date','Use_Promotion','Previous_Cancellations','Visted_Previously'],axis=1)

In [36]:
y_test = xgb.predict(x_test)

In [48]:
test_df.tail()

,Reservation_Status,Reservation-id
4313,2,39110574
4314,2,5496554
4315,2,59004046
4316,2,65838682
4317,1,14474101


In [37]:
test_df = pd.DataFrame(y_test, columns = ['Reservation_Status'])

In [38]:
test_df['Reservation-id'] = test['Reservation-id']

In [39]:
column_titles = ['Reservation-id','Reservation_Status']
test_results1 = test_df.reindex(columns=column_titles)

In [40]:
test_results1.head()

,Reservation-id,Reservation_Status
0,62931593,2
1,70586099,2
2,4230648,2
3,25192322,2
4,80931528,2


In [41]:
test_results1.to_csv('submission_2.csv')

DAY 2 - NAIVE BAYES ALGORITHM

In [14]:
gnb = GaussianNB()

In [15]:
gnb.fit(x_train, y_train)

GaussianNB()

In [16]:
y_pred_NB = gnb.predict(x_val)

In [17]:
nb_fscore = f1_score(y_val, y_pred_NB, average='weighted')

In [18]:
nb_fscore

0.43263349387055267

In [57]:
#Adaboost
ada = AdaBoostClassifier()
ada.fit(x_train, y_train)

y_ada_pred = ada.predict(x_val)

In [58]:
print(f1_score(y_val,y_ada_pred,average='weighted'))

0.03662053266502548


In [ ]:
#KNN Classification

In [59]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train, y_train)

KNeighborsClassifier()

In [60]:
y_pred_knn = classifier.predict(x_val)

In [61]:
print(f1_score(y_val, y_pred_knn, average='weighted'))

0.1141185455773966
